# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

### Importing Libraries

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#imports to help me plot my venn diagrams
import matplotlib_venn as venn2
from matplotlib_venn import venn2
from pylab import rcParams

# import the util.py file where I define my functions
from utils import *

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [2]:
# load in the data
'''
There are 2 warnings when we read in the datasets:
DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
interactivity=interactivity, compiler=compiler, result=result)

This warning happens when pandas attempts to guess datatypes on particular columns, I will address this on 
the pre-processing steps
'''
azdias = pd.read_csv(r"C:\Users\sousa\Desktop\github\Arvato\data\azdias.csv")
customers = pd.read_csv(r"C:\Users\sousa\Desktop\github\Arvato\data\customers.csv")
attributes = pd.read_csv(r"C:\Users\sousa\Desktop\github\Arvato\data\features.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# I will now check what is the problem with the columns 19 and 20
# getting the name of these columns
print(azdias.iloc[:,19:21].columns)
print(customers.iloc[:,19:21].columns)

Index(['CAMEO_DEUG_2015', 'CAMEO_INTL_2015'], dtype='object')
Index(['CAMEO_DEUG_2015', 'CAMEO_INTL_2015'], dtype='object')


In [4]:
# checking the unique values in these columns for possible issues
print(azdias.CAMEO_DEUG_2015.unique())
print(azdias.CAMEO_INTL_2015.unique())
print(customers.CAMEO_DEUG_2015.unique())
print(customers.CAMEO_INTL_2015.unique())

[nan 8.0 4.0 2.0 6.0 1.0 9.0 5.0 7.0 3.0 '4' '3' '7' '2' '8' '9' '6' '5'
 '1' 'X']
[nan 51.0 24.0 12.0 43.0 54.0 22.0 14.0 13.0 15.0 33.0 41.0 34.0 55.0 25.0
 23.0 31.0 52.0 35.0 45.0 44.0 32.0 '22' '24' '41' '12' '54' '51' '44'
 '35' '23' '25' '14' '34' '52' '55' '31' '32' '15' '13' '43' '33' '45'
 'XX']
[1.0 nan 5.0 4.0 7.0 3.0 9.0 2.0 6.0 8.0 '6' '3' '8' '9' '2' '4' '1' '7'
 '5' 'X']
[13.0 nan 34.0 24.0 41.0 23.0 15.0 55.0 14.0 22.0 43.0 51.0 33.0 25.0 44.0
 54.0 32.0 12.0 35.0 31.0 45.0 52.0 '45' '25' '55' '51' '14' '54' '43'
 '22' '15' '24' '35' '23' '12' '44' '41' '52' '31' '13' '34' '32' '33'
 'XX']


### It seems like the mixed type issue comes from that random X that appears in these columns, there are ints, floats and strings all in the mix

In [5]:
cols = ['CAMEO_DEUG_2015', 'CAMEO_INTL_2015']
azdias = mixed_type_fixer(azdias, cols)
customers = mixed_type_fixer(customers, cols)

#checking if values were fixed
azdias.CAMEO_DEUG_2015.unique()
customers.CAMEO_INTL_2015.unique()

In [6]:
#doing a quick check of categorical features and see if some are too granular to be maintained
tst = categorical_checker(azdias, attributes)

AGER_TYP                      5
ANREDE_KZ                     2
CAMEO_DEU_2015               45
CAMEO_DEUG_2015               9
CJT_GESAMTTYP                 6
D19_BANKEN_DATUM             10
D19_BANKEN_OFFLINE_DATUM     10
D19_BANKEN_ONLINE_DATUM      10
D19_GESAMT_DATUM             10
D19_GESAMT_OFFLINE_DATUM     10
D19_GESAMT_ONLINE_DATUM      10
D19_KONSUMTYP                 7
D19_TELKO_DATUM              10
D19_TELKO_OFFLINE_DATUM      10
D19_TELKO_ONLINE_DATUM       10
D19_VERSAND_DATUM            10
D19_VERSAND_OFFLINE_DATUM    10
D19_VERSAND_ONLINE_DATUM     10
D19_VERSI_DATUM              10
D19_VERSI_OFFLINE_DATUM      10
D19_VERSI_ONLINE_DATUM       10
FINANZTYP                     6
GEBAEUDETYP                   7
GFK_URLAUBERTYP              12
GREEN_AVANTGARDE              2
KBA05_BAUMAX                  6
KK_KUNDENTYP                  6
LP_FAMILIE_FEIN              12
LP_FAMILIE_GROB               6
LP_STATUS_FEIN               10
LP_STATUS_GROB                5
NATIONAL

### Based on the categorical info it might be a good idea do drop CAMEO_DEU_2015 column, it is far too fragmented with 45 different category values

In [17]:
def feat_fixer(df, attributes_df):
    df.replace({'X': np.nan, 'XX':np.nan, '': np.nan, ' ':np.nan})
    #parsing unknown and missing values from the attributes dataframe
    m_o_u_list = [x.replace("[","").replace("]","").split(',') for x in attributes_df['missing_or_unknown']]
    
    #changing strings to floats
    m_o_u_float = []
    for x in m_o_u_list:
        #list inside list
        list_of_list = []
        for missing in x:
            try:
                missing = float(missing)
                list_of_list.append(missing)
            except:
                missing = np.nan
                list_of_list.append(missing)
        list_of_list
                
        m_o_u_float.append(list_of_list)
        
        #replacing the missing and unknown values with nan
    for col, m_unknown in zip(df.columns, m_o_u_float):
        for miss in m_unknown:
            df[col].replace(m_unknown, np.nan, inplace = True)
        
    return df
    

In [18]:
tester = feat_fixer(azdias, attributes)
tester.head(30)


,Unnamed: 0,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,NaN,910215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
1,1.0,910220,NaN,9.0,NaN,NaN,NaN,NaN,NaN,21.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
2,2.0,910225,NaN,9.0,17.0,NaN,NaN,NaN,NaN,17.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
3,3.0,910226,2.0,1.0,13.0,NaN,NaN,NaN,NaN,13.0,...,0.0,7.0,10.0,11.0,NaN,9.0,7.0,3,2,4
4,4.0,910241,NaN,1.0,20.0,NaN,NaN,NaN,NaN,14.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3
5,5.0,910244,3.0,1.0,10.0,NaN,NaN,NaN,NaN,10.0,...,2.0,10.0,7.0,4.0,6.0,9.0,7.0,4,2,1
6,6.0,910248,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,7.0,10.0,10.0,3.0,9.0,5.0,4,2,2
7,7.0,910261,NaN,1.0,14.0,NaN,NaN,NaN,NaN,14.0,...,2.0,10.0,12.0,9.0,5.0,9.0,1.0,1,1,1
8,8.0,645145,NaN,9.0,16.0,NaN,NaN,NaN,NaN,16.0,...,4.0,8.0,11.0,8.0,5.0,8.0,1.0,6,1,3
9,9.0,645153,NaN,5.0,17.0,NaN,NaN,NaN,NaN,17.0,...,4.0,1.0,1.0,1.0,4.0,3.0,7.0,4,2,3


In [9]:
print(attributes_list)

[[-1.0, 0.0]]


In [ ]:
attributes.missing_or_unknown.unique()

In [ ]:
cols = ['missing_or_unknown']

attributes[cols] = attributes[cols].replace({'X': np.nan, 'XX':np.nan})

In [ ]:
#to facilitate deletion and manipulation of rows based on row labels I will set attributes as indexes
attributes = attributes.set_index('attribute')

#dropping the rows that are problematic (too many feature categories)
attributes = attributes.drop('CAMEO_DEU_2015', axis = 0)

In [ ]:
azdias = azdias.drop(['CAMEO_DEU_2015'], axis = 1)
customers = customers.drop(['CAMEO_DEU_2015'], axis = 1)

### There is an extra column called Unnamed that seems like an index duplication, I will now drop it

In [ ]:
#dropping unnamed column
azdias = azdias.drop(azdias.columns[0], axis = 1)
customers = customers.drop(customers.columns[0], axis = 1)

In [ ]:
# checking how the azdias dataframe looks like
print('Printing dataframe shape')
print(azdias.shape)
print('________________________________________________________')

azdias.head()

In [ ]:
# checking how the customer dataframe looks like
print('Printing dataframe shape')
print(customers.shape)
print('________________________________________________________')

customers.head()

In [ ]:
# Check the summary csv file
print(attributes.shape)
attributes.head()

### On the dataframe shapes:

#### For now it is noted that the 2 initial working dataframes had shape discrepancy between them (rows, columns):
#### azdias: (891221, 365)
#### customers: (191652, 368)

In [ ]:
#saving the unique attribute names to lists
attributes = attributes.reset_index() 
attributes_list = attributes.attribute.unique().tolist()
azdias_list = list(azdias.columns)
customers_list = list(customers.columns)

In [ ]:
#establishing uniqueness of the attributes accross the datasets in work
common_to_all = (set(attributes_list) & set(azdias_list) & set(customers_list))
unique_to_azdias = (set(azdias_list) - set(attributes_list) - set(customers_list))
unique_to_customers = (set(customers_list) - set(attributes_list) - set(azdias_list))
unique_to_attributes = (set(attributes_list) - set(customers_list) - set(azdias_list))
unique_to_attributes_vs_azdias = (set(attributes_list) - set(azdias_list))
unique_to_azdias_vs_attributes = (set(attributes_list) - set(azdias_list))
common_azdias_attributes = (set(azdias_list) & set(attributes_list))

print("No of items common to all 3 daframes: " + str(len(common_to_all)))
print("No of items exclusive to azdias: " + str(len(unique_to_azdias)))
print("No of items exclusive to customers: " + str(len(unique_to_customers)))
print("No of items exclusive to attributes: " + str(len(unique_to_attributes)))
print("No of items overlapping between azdias and attributes: " + str(len(common_azdias_attributes)))
print("No of items exclusive to attributes vs azdias: " + str(len(unique_to_attributes_vs_azdias)))
print("No of items exclusive to azdias vs attributes: " + str(len(unique_to_azdias_vs_attributes)))

In [ ]:
rcParams['figure.figsize'] = 8, 8

ax = plt.axes()
ax.set_facecolor('lightgrey')
v = venn2([len(azdias_list), len(attributes_list), len(common_azdias_attributes)], 
      set_labels=('Azdias', 'Attributes'),
         set_colors = ['cyan', 'grey']);

plt.title("Attribute presence on Azdias vs DIAS Attributes ")
plt.show()

### From this little exploration we got quite a little bit of information:
#### - There are 3 extra features in the customers dataset, it corresponds to the columns 'CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'

#### - All the datasets share 327 features between them

#### - The attributes file has 5 columns corresponding to feature information that does not exist in the other datasets

## Preprocessing
### Now that I have a birds-eye view of the data I will proceed with cleaning and handling missing calues, re-encode features (since the first portion of this project will involve unsupervised learning), perform some feature enginnering and scaling.

### Assessing missing data and replacing it with nan

### checking columns missing more than 30% of the data

In [ ]:
percent_missing_azdias_df = percentage_of_missing(azdias)
percent_missing_azdias_df.head()

In [ ]:
percent_missing_customers_df = percentage_of_missing(customers)
percent_missing_customers_df.head()

In [ ]:
print('Empty cells in Azdias dataframe: ', azdias.isnull().sum().sum())
print('Empty cells in Customers dataframe: ', customers.isnull().sum().sum())
print('________________________________________________________')
print("Number of Azdias columns not missing values: ", ((percent_missing_azdias_df['percent_missing'] == 0.0).sum()))
print("Number of Customers columns not missing values: ", ((percent_missing_customers_df['percent_missing'] == 0.0).sum()))

In [ ]:
azdias_missing_over_30 = split_on_percentage(percent_missing_azdias_df, 30, '>')
azdias_missing_less_30 = split_on_percentage(percent_missing_azdias_df, 30, '<=')
print(azdias_missing_over_30.head())
print(azdias_missing_less_30.head())

In [ ]:
customers_missing_over_30 = split_on_percentage(percent_missing_customers_df, 30, '>')
customers_missing_less_30 = split_on_percentage(percent_missing_customers_df, 30, '<=')
print(customers_missing_over_30.head())
print(customers_missing_less_30.head())

In [ ]:
figure, axes = plt.subplots(4, 1, figsize = (15,15), squeeze = False)

azdias_missing_over_30.sort_values(by = 'percent_missing', ascending = False).plot(kind = 'bar', x = 'column_name', y = 'percent_missing',
                                                                                ax = axes[0][0], title = 'Azdias percentage of missing values over 30%' )

#due to the sheer amount of data points to be plotted this does not make an appealing vis so I will restrict
#the number of plotted points to 40
azdias_missing_less_30.sort_values(by = 'percent_missing', ascending = False)[:40].plot(kind = 'bar', x = 'column_name', y = 'percent_missing',
                                                                                ax = axes[1][0], title = 'Azdias percentage of missing values less 30%' )

customers_missing_over_30.sort_values(by = 'percent_missing', ascending = False).plot(kind = 'bar', x = 'column_name', y = 'percent_missing',
                                                                                ax = axes[2][0], title = 'Customers percentage of missing values over 30%' )

#due to the sheer amount of data points to be plotted this does not make an appealing vis so I will restrict
#the number of plotted points to 40
customers_missing_less_30.sort_values(by = 'percent_missing', ascending = False)[:40].plot(kind = 'bar', x = 'column_name', y = 'percent_missing',
                                                                                ax = axes[3][0], title = 'Customers percentage of missing values less 30%' )

plt.tight_layout()
plt.show()

### The vast majority of the columns with missing values have a percent of missing under 30% which might means there is some recoverable data.
### Based on this information I will remove columns with more than 30% missing values

In [ ]:
#extracting column names with more than 30% values missing so we can drop them from azdias df
azdias_col_delete = columns_to_delete(azdias_missing_over_30)

#extracting column names with more than 30% values missing so we can drop them from customers df
customers_col_delete = columns_to_delete(customers_missing_over_30)

In [ ]:
#dropping the columns identified in the previous lists

azdias = azdias.drop(azdias_col_delete, axis = 1)
customers = customers.drop(customers_col_delete, axis = 1)

### Now that we dropped columns missing more than 30% of their data it is also important to check the rows (individuals) that have far too much missing information

In [ ]:
def rows_to_delete(df, boundary):
    '''
    This function identifies rows missing more than a particular percentage of data specified with boundary
    Args: 
    dataframe: already cleaned up of columns missing more than a boundary defined percentage 
    boundary: percentage of missing values
    
    returns:
    dataframe with dropped rows with more than a percentage of missing values
    '''
    rows_to_drop = list(df[df.isnull().sum(axis = 1) > boundary].index)
    print('Rows to drop from this dataframe: ' + str(len(rows_to_drop)))
    df_clean_rows = df.drop(rows_to_drop, axis = 0)
    
    return df_clean_rows  

In [ ]:
azdias_3 = rows_to_delete(azdias_2, 30)
azdias_3.head()

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')